In [ ]:
# BEA data

In [13]:
from pySmartDL import SmartDL
from os.path import basename
import tarfile
import os
from pymongo import MongoClient
import json
import requests
import pandas as pd
from io import StringIO
import numpy as np
from datetime import datetime

from io import BytesIO
from zipfile import ZipFile
import re

In [ ]:
url = 'https://apps.bea.gov/regional/zip/CAINC1.zip'
response = requests.get(url)

In [32]:
def gen_lst_value(filename):
    pattern = r'\d{4}'
    years = re.findall(pattern, filename)
    # 
    lst     = []
    start   = int(years[0])
    end     = int(years[1]) + 1
    for year in range(start, end) :
        lst += [f'{year}']
    return lst 



# Create a BytesIO object and write the content of the response to it
with BytesIO(response.content) as zip_file:
    # Seek to the start of the file
    zip_file.seek(0)

    # Unzip the file in memory
    with ZipFile(zip_file) as zf:
        # Get the name of the first file in the zip
        # filename = zf.namelist()[0]
        filename = next((name for name in zf.namelist() if "ALL_AREAS" in name), None)

        # Read the file into a pandas DataFrame
        # Here I am assuming it's a CSV for demonstration purposes
        with zf.open(filename) as file:
            df = pd.read_csv(file, encoding='latin-1')  # If it's actually a .dta file, replace with pd.read_stata(file)

# use the df 
df['fips'] = df['GeoFIPS'].str.replace('\"','').str.strip()
df['fipsstr'] = df['fips'].str.slice(0,2)
df['fips.county'] = df['fips'].str.slice(2,5)
df['fips.county.name'] = df['GeoName'].str.split(',',expand=True)[0].str.strip()
df['fips.state'] = df['GeoName'].str.split(',',expand=True)[1].str.strip()
# drop aggreaged county
df = df.loc[df['fips.county'] != '000']

df['Unit'][df['LineCode'] == 3] = 'personal_income_per_cap'
df['Unit'][df['LineCode'] == 2] = 'population'

df = df.dropna(subset='GeoName').drop(columns=['GeoFIPS','Region', 'TableName','IndustryClassification',])
# 
# 
df

df_melt = df.melt(id_vars   =['fipsstr', 'fips', 'fips.state', 'fips.county', 'fips.county.name', 'Unit','LineCode'], 
             value_vars     = gen_lst_value(filename),
             var_name       = 'year')

df_melt = df_melt.loc[df_melt['LineCode'] != 1].reset_index(drop=True)
df_melt

df_reg_income = df_melt.pivot(index=['fipsstr', 'fips', 'fips.state', 'fips.county', 'fips.county.name','year'], columns=['Unit'], values='value').reset_index()
df_reg_income['personal_income_per_cap'] = df_reg_income['personal_income_per_cap'].replace(r'(NA)',np.NaN).astype('float64')
df_reg_income['population'] = df_reg_income['population'].replace(r'(NA)',np.NaN).astype('float64')
df_reg_income = df_reg_income.reset_index(drop=True)

df_reg_income


C:\Users\SeanJ\AppData\Local\Temp\ipykernel_16000\2003354196.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Unit'][df['LineCode'] == 2] = 'population'


Unit,fipsstr,fips,fips.state,fips.county,fips.county.name,year,personal_income_per_cap,population
0,01,01001,AL,001,Autauga,1969,2780.0,25166.0
1,01,01001,AL,001,Autauga,1970,3158.0,24606.0
2,01,01001,AL,001,Autauga,1971,3454.0,25508.0
3,01,01001,AL,001,Autauga,1972,3687.0,27166.0
4,01,01001,AL,001,Autauga,1973,4039.0,28463.0
...,...,...,...,...,...,...,...,...
166415,56,56045,WY,045,Weston,2017,42555.0,7008.0
166416,56,56045,WY,045,Weston,2018,44802.0,6947.0
166417,56,56045,WY,045,Weston,2019,46281.0,6938.0
166418,56,56045,WY,045,Weston,2020,47499.0,6809.0
